### PROJECT: Rating Product & Sorting Reviews in Amazon

###################################################
#### Business Problem
###################################################

One of the most important problems in e-commerce is the accurate calculation of post-purchase ratings for products. The solution to this problem means greater customer satisfaction for the e-commerce site, better visibility for sellers' products, and a seamless shopping experience for buyers. Another problem is the proper sorting of product reviews. Misleading reviews can directly impact the sale of a product, resulting in both financial loss and customer loss. By solving these two fundamental problems, e-commerce sites and sellers can increase their sales, while customers can complete their purchasing journey seamlessly.

###################################################
#### The story of Dataset
###################################################

This dataset containing Amazon product data includes various metadata with product categories. The product with the most reviews in the Electronics category has user ratings and reviews.

Variables:
reviewerID: User ID
asin: Product ID
reviewerName: User name
helpful: Helpful review rating
reviewText: Review
overall: Product rating
summary: Review summary
unixReviewTime: Review time
reviewTime: Raw review time
day_diff: Number of days since the review was posted
helpful_yes: Number of people who found the review helpful
total_vote: Total number of votes given to the review"

###################################################
#### TASK 1: Calculate the Average Rating Based on Recent Reviews and Compare it with the Existing Average Rating.

In the provided dataset, users have given ratings and reviews for a product. The aim of this task is to evaluate the given ratings by weighting them according to the date. A comparison needs to be made between the initial average rating and the weighted rating obtained based on the date.

In [3]:
import pandas as pd
import datetime as dt

df = pd.read_csv("amazon_review.csv")
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.0,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.0,best deal around,1373673600,2013-07-13,513,0,0


In [4]:
# Step 1: Calculate the average rating of the product.

df.groupby("asin").agg({"overall": "mean"})

,overall
asin,
B007WTAJTO,4.587589


In [5]:
# Adım 2: Tarihe göre ağırlıklı puan ortalamasını hesaplayınız.

def weighted_avg_for_date(x=0.4, y=0.35, z=0.25):
    return df[df["day_diff"] <= 60]["overall"].mean() * x + \
        df[(df["day_diff"] > 60) & (df["day_diff"] <= 150)]["overall"].mean() * y + \
            df[df["day_diff"] <= 150]["overall"].mean() * z

weighted_avg_for_date(0.45,0.30,0.25)

4.740141795485782

In [6]:
# Step 3: Compare and interpret the average of each time period in the weighted rating.

In [7]:
_60_day = df[df["day_diff"] <= 60]["overall"].mean() 
_60_150_day = df[(df["day_diff"] > 60) & (df["day_diff"] <= 150)]["overall"].mean() 
_150_day = df[df["day_diff"] <= 150]["overall"].mean() 

print(f"60< : {_60_day} \n"
      f"60<_<150 : {_60_150_day} \n"
      f"+150 : {_150_day}")

60< : 4.770491803278689 
60<_<150 : 4.705128205128205 
+150 : 4.727528089887641


###################################################
####
#### Task 2: Determine the 20 reviews that will be displayed on the product detail page for the product

Step 1: Generate the 'helpful_no' variable.

* 'total_vote' is the total number of up-down votes given to a review.
* 'up' means 'helpful'.
* There is no 'helpful_no' variable in the dataset and it needs to be generated based on the existing variables.
* Subtract the number of helpful votes (helpful_yes) from the total number of votes (total_vote) to find the number of unhelpful votes (helpful_no).

In [8]:
# helpful_no 
df[['helpful_yes', 'helpful_no']] = df['helpful'].str.strip('[]').str.split(',', expand=True).astype(int)

df["helpful_no"].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    1
9    0
Name: helpful_no, dtype: int32

In [9]:
print(df[df["helpful_no"] > 0].shape)
print(df[df["helpful_yes"] > 0].shape)

(555, 13)
(413, 13)


In [10]:
# total-vote

df["total_vote"] = df["helpful_yes"] + df["helpful_no"]

Step 2: Calculate score_pos_neg_diff, score_average_rating, and wilson_lower_bound scores and add them to the data.

Calculate score_pos_neg_diff, score_average_rating, and wilson_lower_bound scores by defining score_pos_neg_diff, score_average_rating, and wilson_lower_bound functions.

* Create scores according to score_pos_neg_diff. Then, save it as score_pos_neg_diff in df.
* Create scores according to score_average_rating. Then, save it as score_average_rating in df.
* Create scores according to wilson_lower_bound. Then, save it as wilson_lower_bound in df.

In [11]:
# avarage rating

df["avg_rating"] = df["helpful_yes"] / df["total_vote"]

In [12]:
# score_pos_neg_diff

df["score_pos_neg_diff"] = df["helpful_yes"] - df["helpful_no"]

In [13]:
# wilson_lower_bound
import math
import scipy.stats as st

def wilson_lower_bound(up, down, confidence=0.95):
    """
    Calculate Wilson Lower Bound Score

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
    - The calculated score is used for product ranking.
    - Note:
        If the scores are between 1-5, they are marked as negative for 1-3 and positive for 4-5 and can be made compatible with the Bernoulli distribution.
        However, this brings some problems along. Therefore, it is necessary to do Bayesian average rating."

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

Step 3: Determine 20 reviews and interpret the results.

In [15]:
df.sort_values("wilson_lower_bound", ascending=False).head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,avg_rating,score_pos_neg_diff,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,3972,2020,0.491440,-68,0.475909
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,2933,1505,0.486874,-77,0.468814
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,3262,1694,0.480687,-126,0.463574
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,917,495,0.460196,-73,0.428170
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.0,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,251,139,0.446215,-27,0.385989
